In [1]:
import os
import matplotlib
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import scipy.misc
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import zipfile
from sklearn import preprocessing
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Add, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Concatenate
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input


from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

In [0]:
train = pd.read_csv("./gdrive/My Drive/mafat/train.csv")
train.set_index('image_id',inplace=True)

In [0]:

test = pd.read_csv("./gdrive/My Drive/mafat/test.csv")
test.set_index('image_id',inplace=True)


In [5]:
k=train.loc[[16490],['p1_x','p_1y',' p2_x',' p2_y',' p3_x',' p3_y',' p4_x',' p4_y']]
k.head()

,p1_x,p_1y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y
image_id,,,,,,,,
16490,3988.399902,1227.588745,3986.710693,1213.288208,4002.600098,1211.411255,4004.289307,1225.711792
16490,3380.804152,33.370945,3329.811654,128.465062,3358.064254,142.935906,3409.056752,44.396350
16490,529.397072,811.912246,535.035063,838.222871,634.639572,814.418020,626.495807,784.348734
16490,1076.361206,858.714233,1070.904541,827.995117,1215.638794,802.285767,1221.095459,833.004883
16490,2588.353180,914.025156,2593.176795,887.150731,2810.928540,932.630526,2807.483101,965.706741


In [6]:
train.head()

,tag_id,p1_x,p_1y,p2_x,p2_y,p3_x,p3_y,p4_x,p4_y,general_class,...,enclosed_cab,spare_wheel,wrecked,flatbed,ladder,enclosed_box,soft_shell_box,harnessed_to_a_cart,ac_vents,color
image_id,,,,,,,,,,,,,,,,,,,,,
16490,15036,3988.399902,1227.588745,3986.710693,1213.288208,4002.600098,1211.411255,4004.289307,1225.711792,small vehicle,...,0,0,0,0,0,0,0,0,-1,black
16490,31658,3380.804152,33.370945,3329.811654,128.465062,3358.064254,142.935906,3409.056752,44.396350,large vehicle,...,-1,-1,0,-1,0,1,0,0,-1,red
17122,26971,3948.334473,1426.797241,3958.876221,1410.750122,4000.665527,1438.202759,3990.123779,1454.249878,small vehicle,...,0,0,0,-1,-1,-1,-1,-1,-1,red
12193,19301,1977.798007,1009.010733,1983.435998,992.096760,2034.177918,1009.010733,2027.913483,1027.177593,small vehicle,...,0,0,0,-1,-1,-1,-1,-1,-1,silver/grey
12193,35906,3196.620177,1284.240545,3193.174738,1311.804057,3288.957942,1321.451286,3292.403381,1293.198686,large vehicle,...,-1,-1,0,1,0,0,0,0,-1,red


In [0]:
image_size=64
num_channels=3
num_pictures=275
train_dir="./gdrive/My Drive/mafat/training imagery/"
val_dir="./gdrive/My Drive/mafat/test imagery/"


In [8]:
image_name=[]
num_tiff=0
for file in os.listdir(train_dir):
  if file[-3:]!='jpg':
    num_tiff=num_tiff+1
    continue
  image_name.append(file[:-4])
print(num_tiff)

80


In [0]:
total_pictures=len(image_name)

In [0]:
#importing data

def get_next_batch(image_index):
  
  images=np.ndarray(shape=(num_pictures,image_size,image_size,num_channels))
  current_index=0
  labels=[]
  num_tiff=0

  while current_index<num_pictures and image_index<total_pictures:
    
      file=image_name[image_index]
    
      image=os.path.join(train_dir,file+'.jpg')
      file_data=mpimg.imread(image)
      height=file_data.shape[0]
      breadth=file_data.shape[1]
    
      image_id=np.ndarray(shape=(1),dtype='int32')
      image_id=int(file)
    
    
      lb=train.loc[image_id,['sub_class']]
      label_gate=0
      if(lb.shape[0]==1):
          lb=lb.values
          label_gate=1
      else:
          lb=lb['sub_class'].tolist()
    
    
      bb=train.loc[image_id,['p1_x','p_1y',' p2_x',' p2_y',' p3_x',' p3_y',' p4_x',' p4_y']].values
      n=bb.shape[0]
      if len(bb.shape)==1:
        box=np.ndarray(shape=(1,8))
        box[0,:]=bb[0]
        bb=box
        n=1
    
      box=np.ndarray(shape=(n,4))
      for i in range(n):
        x1=min(bb[i,0],bb[i,2],bb[i,4],bb[i,6])
        x2=max(bb[i,0],bb[i,2],bb[i,4],bb[i,6])
        y1=min(bb[i,1],bb[i,3],bb[i,5],bb[i,7])
        y2=max(bb[i,1],bb[i,3],bb[i,5],bb[i,7])
        box[i,0]=y1/height
        box[i,1]=x1/breadth
        box[i,2]=y2/height
        box[i,3]=x2/breadth
        y=np.ndarray(shape=(1,file_data.shape[0],file_data.shape[1],3))
        y[0,:,:,:]=file_data[:,:,0:3]
   
      cropped=tf.image.crop_and_resize(image=y,boxes=box,box_ind=[0]*box.shape[0],crop_size=[image_size,image_size],method='bilinear')
      sess=tf.Session()
      with sess.as_default():
          tensor=cropped
          cropped=tensor.eval()
      n=cropped.shape[0]
    
    
      for i in range(n):
          images[current_index,:,:,:]=cropped[i,:,:,:]
          
          current_index=current_index+1
          if label_gate==0:
            labels.append(lb[i])
          else:
            labels.extend(lb)

          if current_index>=num_pictures:
            break
      image_index=image_index+1
  
  labels=np.asarray(labels)
  #shuffling data

  random=np.arange(num_pictures)
  np.random.shuffle(random)
  images=images[random]
  labels=labels[random]
  
  
  return images[:250,:,:,:],labels[:250],image_index,images[250:,:,:,:],labels[250:]




In [0]:
all_labels=train['sub_class'].values
all_labels=np.asarray(all_labels)
le = preprocessing.LabelEncoder()
training_le = le.fit(all_labels)
num_classes=le.classes_.shape[0]

In [0]:
def get_encoded_label(labels,val_y):
  return training_le.transform(labels),training_le.transform(val_y)

In [13]:
le.classes_

array(['bus', 'cement mixer', 'crane truck',
       'dedicated agricultural vehicle', 'hatchback', 'jeep',
       'light truck', 'minibus', 'minivan', 'pickup', 'prime mover',
       'sedan', 'tanker', 'truck', 'van'], dtype=object)

In [0]:
def identity_block(X, f, filters):
    
    F1, F2, F3 = filters
     
    X_shortcut = X
   
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = X = Conv2D(filters=F3,kernel_size=(1,1),strides=(1,1),padding='valid',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    X = layers.Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [0]:

def convolutional_block(X, f, filters,s = 2):
    
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides = (s,s), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2,(f,f),strides=(1,1),padding='same',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    X = Conv2D(F3,(1,1),strides=(1,1),padding='valid',kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)

    X_shortcut = Conv2D(F3,(1,1),strides=(s,s),padding='valid',kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    X = layers.Add()([X,X_shortcut])
    X = Activation('relu')(X)
    return X

In [0]:

def ResNet50(input_shape, classes):
    
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)
    
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    X = convolutional_block(X,f=3,filters=[128,128,512],s=2)
    X = identity_block(X,3,[128,128,512])
    X = identity_block(X,3,[128,128,512])
    X = identity_block(X,3,[128,128,512])

    X = convolutional_block(X,f=3,filters=[256,256,1024],s=2)
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])
    X = identity_block(X,3,[256,256,1024])

    X = convolutional_block(X,f=3,filters=[512,512,2048],s=2)
    X = identity_block(X,3,[512,512,2048])
    X = identity_block(X,3,[512,512,2048])

    X = AveragePooling2D((2,2))(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X)

    return model

In [0]:
model = ResNet50(input_shape = (image_size, image_size, 3), classes=num_classes)

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
num_times=2
image_index=0
X_val=np.ndarray(shape=(0,image_size,image_size,num_channels))
Y_val=np.ndarray(shape=(0))

for i in range(num_times):
  
  X,labels,image_index,val_x,val_y=get_next_batch(image_index)
  print('image_index',image_index)
  Y,val_y=get_encoded_label(labels,val_y)
  X_val=np.concatenate((X_val,val_x), axis=0)
  Y_val=np.concatenate((Y_val,val_y), axis=0)
  model.fit(X, Y, epochs = 2, batch_size = 32)

image_index 63
Epoch 1/2
250/250 [==============================] - 10s 38ms/step - loss: 1.5247 - acc: 0.5680
Epoch 2/2
250/250 [==============================] - 2s 7ms/step - loss: 1.4144 - acc: 0.5880
image_index 131
Epoch 1/2
250/250 [==============================] - 2s 10ms/step - loss: 1.6567 - acc: 0.5720
Epoch 2/2
250/250 [==============================] - 2s 7ms/step - loss: 1.5743 - acc: 0.5760


In [19]:
val_set = model.evaluate(X_val, Y_val)
print ("Loss = " + str(val_set[0]))
print ("Test Accuracy = " + str(val_set[1]))

50/50 [==============================] - 1s 23ms/step
Loss = 2.192920112609863
Test Accuracy = 0.4799999976158142


In [0]:
prob=model.predict(X_val)

In [0]:
model.save("./gdrive/My Drive/mafat/model")

In [0]:
model=load_model("./gdrive/My Drive/mafat/model")

In [0]:
model.save("./gdrive/My Drive/mafat/MyModel")

In [0]:
mymodel=load_model("./gdrive/My Drive/mafat/MyModel")

In [9]:
val_image_name=[]
tiff=0
tif=0
jpg=0
for file in os.listdir(val_dir):

  if file[-3:]=='tif':
    tif+=1
    val_image_name.append(file)
  if file[-3:]=='iff':
    tiff+=1
    val_image_name.append(file)
    
  if file[-3:]=='jpg':
    jpg+=1
    val_image_name.append(file)
    
print('tif=',tif)
print('tiff=',tiff)
print('jpg=',jpg)

tif= 9
tiff= 2
jpg= 2531


In [0]:
import gc


In [0]:
#importing data


def get_next_val_batch(image_index,num_pictures,last_index):
  
  images=np.ndarray(shape=(num_pictures,image_size,image_size,num_channels))
  current_index=0
  tag_id=[]
  
   

  while current_index<num_pictures and image_index<len(val_image_name):
    
      file=val_image_name[image_index]
      image=os.path.join(val_dir,file)
      
      file_data=mpimg.imread(image)
      height=file_data.shape[0]
      breadth=file_data.shape[1]
    
      image_id=np.ndarray(shape=(1),dtype='int32')
      if file[-3:]=='iff':
        file=file[:-5]
      else:
        file=file[:-4]
      
      image_id=int(file)
    
    
      lb=test.loc[image_id,['tag_id']]
      label_gate=0
      if(lb.shape[0]==1):
          lb=lb.values
          label_gate=1
      else:
          lb=lb['tag_id'].tolist()
    
    
      bb=test.loc[image_id,['p1_x','p_1y',' p2_x',' p2_y',' p3_x',' p3_y',' p4_x',' p4_y']].values
      n=bb.shape[0]
      if len(bb.shape)==1:
        box=np.ndarray(shape=(1,8))
        box[0,:]=bb[0]
        bb=box
        n=1
    
      box=np.ndarray(shape=(n,4))
      for i in range(n):
        x1=min(bb[i,0],bb[i,2],bb[i,4],bb[i,6])
        x2=max(bb[i,0],bb[i,2],bb[i,4],bb[i,6])
        y1=min(bb[i,1],bb[i,3],bb[i,5],bb[i,7])
        y2=max(bb[i,1],bb[i,3],bb[i,5],bb[i,7])
        box[i,0]=y1/height
        box[i,1]=x1/breadth
        box[i,2]=y2/height
        box[i,3]=x2/breadth
        y=np.ndarray(shape=(1,file_data.shape[0],file_data.shape[1],3))
        y[0,:,:,:]=file_data[:,:,0:3]
   
      cropped=tf.image.crop_and_resize(image=y,boxes=box,box_ind=[0]*box.shape[0],crop_size=[image_size,image_size],method='bilinear')
      sess=tf.Session()
      with sess.as_default():
          tensor=cropped
          cropped=tensor.eval()
      n=cropped.shape[0]
      gate=0
    
      for i in range(last_index+1,n):
          images[current_index,:,:,:]=cropped[i,:,:,:]
          
          current_index=current_index+1
          if label_gate==0:
            tag_id.append(lb[i])
          else:
            tag_id.extend(lb)
          if current_index%200==0:
            print(current_index)
          if current_index>=num_pictures:
            last_index=i
            gate=1
            break
            
      if last_index!=-1 and gate==0:
        last_index=-1
      image_index=image_index+1
      del file,image,file_data,image_id,lb,bb,sess,cropped,box
      gc.collect()

  tag_id=np.asarray(tag_id)
 
  if last_index!=-1:
    image_index-=1
  
  return images,tag_id,image_index,last_index




In [12]:

val_image_index=2520
last_index=2
for i in range(1):
  X_val,tag_id,val_image_index,last_index=get_next_val_batch(val_image_index,379,last_index)
  print('val_image_index',val_image_index)
  print('last_index',last_index)
  prob=mymodel.predict(X_val)
  df1=pd.DataFrame(data=prob, index=None,dtype='float32', columns=['bus', 'cement mixer', 'crane truck','dedicated agricultural vehicle', 'hatchback', 'jeep','light truck', 'minibus', 'minivan', 'pickup', 'prime mover','sedan', 'tanker', 'truck', 'van'])
  df2=pd.DataFrame(data=tag_id,index=None,dtype='int32',columns=['tag_id'])
  df=pd.concat([df2,df1],axis=1)
  df.to_csv("./gdrive/My Drive/mafat/sub_classes_prob60.csv")
  del X_val,tag_id,prob,df1,df2,df
  gc.collect()






200
val_image_index 2541
last_index 19


In [12]:
os.listdir("./gdrive/My Drive/mafat")

['MyModel',
 'train.csv',
 'test.csv',
 'Readme.txt',
 'training imagery',
 '__MACOSX',
 'test imagery',
 'model',
 'sub_classes_prob1.csv',
 'sub_classes_prob2.csv',
 'sub_classes_prob3.csv']

In [13]:
df2 = pd.read_csv("./gdrive/My Drive/mafat/sub_classes_prob3.csv")
df2.shape

(2000, 17)

In [0]:
df1 = pd.read_csv("./gdrive/My Drive/mafat/sub_classes_prob10.csv")
df2 = pd.read_csv("./gdrive/My Drive/mafat/sub_classes_prob20.csv")
df3 = pd.read_csv("./gdrive/My Drive/mafat/sub_classes_prob30.csv")
df4 = pd.read_csv("./gdrive/My Drive/mafat/sub_classes_prob40.csv")
df5 = pd.read_csv("./gdrive/My Drive/mafat/sub_classes_prob50.csv")
df6 = pd.read_csv("./gdrive/My Drive/mafat/sub_classes_prob60.csv")

df_array=[df1,df2,df3,df4,df5,df6]
df=pd.concat(df_array,axis=0)
df.to_csv("./gdrive/My Drive/mafat/sub_classes_prob.csv")

In [0]:
df = pd.read_csv("./gdrive/My Drive/mafat/sub_classes_prob.csv")

In [7]:
df.head()

,Unnamed: 0,Unnamed: 0.1,tag_id,bus,cement mixer,crane truck,dedicated agricultural vehicle,hatchback,jeep,light truck,minibus,minivan,pickup,prime mover,sedan,tanker,truck,van
0,0,0,31467,1.567019e-03,0.012492,1.210401e-03,2.818474e-05,0.526311,0.105909,1.714338e-03,3.440918e-04,0.024826,0.008110,2.325420e-03,0.294714,4.582894e-06,0.011682,0.008762
1,1,1,19629,1.847371e-02,0.046218,2.522349e-03,1.963599e-04,0.445598,0.144789,6.104801e-03,3.782016e-03,0.082189,0.022453,6.979203e-03,0.094188,4.243241e-05,0.043551,0.082913
2,2,2,23792,2.635901e-02,0.038014,2.071212e-03,1.760906e-04,0.460633,0.051653,4.647544e-03,4.159627e-03,0.099698,0.029197,6.527831e-03,0.086820,3.299926e-05,0.034476,0.155536
3,3,3,11188,1.115992e-07,0.000007,2.045982e-07,4.494190e-10,0.000298,0.000005,5.212257e-07,1.450198e-08,0.000093,0.000018,1.289467e-07,0.999567,7.451388e-11,0.000001,0.000008
4,4,4,38668,6.696758e-02,0.060829,3.307592e-03,3.788816e-04,0.161127,0.064813,7.638994e-03,1.072831e-02,0.099589,0.036162,9.958354e-03,0.092572,8.196871e-05,0.087309,0.298538


In [0]:
df = pd.read_csv("./gdrive/My Drive/mafat/answer.csv")

In [0]:
df.to_csv("./gdrive/My Drive/mafat/new/answer.csv")